In [ ]:
import os
import numpy as np
import pyart
import fsspec
import xarray as xr
from datetime import datetime, timedelta
import warnings
from io import BytesIO
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cartopy.crs as ccrs
from metpy.plots import USCOUNTIES
from PIL import Image
from IPython.display import display, Image as IPImage
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Setup the S3 filesystem
fs = fsspec.filesystem("s3", anon=True)

# Define the start and end dates and hours
start_date = datetime(2024, 6, 22, 23, 0)  # YYYY/MM/DD HH
end_date = datetime(2024, 6, 23, 8, 0)   # YYYY/MM/DD HH
station = 'KLOT'
latitude = [41.700937896518866, 41.704120]
longitude = [-87.99578103231573, -87.968328]
labels = ['Tower', 'SMC']
markers = ['^', 'o']
colors = ['magenta', 'cyan']

# Generate the list of files for the specified date and hour range
files = []
current_date = start_date

while current_date <= end_date:
    date_str = current_date.strftime("%Y/%m/%d")
    date_str_compact = current_date.strftime("%Y%m%d")

    if current_date.date() == start_date.date():
        start_hour = start_date.hour
    else:
        start_hour = 0

    if current_date.date() == end_date.date():
        end_hour = end_date.hour
    else:
        end_hour = 23

    for hour in range(start_hour, end_hour + 1):
        hour_str = f"{hour:02d}"
        all_files = fs.glob(f"s3://noaa-nexrad-level2/{date_str}/{station}/{station}{date_str_compact}_{hour_str}*")
        filtered_files = [f for f in all_files if not f.endswith('_MDM')]
        files += sorted(filtered_files)

    current_date = current_date.replace(hour=0) + timedelta(days=1)
    
# Function to check and print the fields in the first radar file
def check_radar_fields(file_path):
    try:
        with fs.open(file_path, 'rb') as f:
            radar_data = f.read()
        radar_file = BytesIO(radar_data)
        radar = pyart.io.read_nexrad_archive(radar_file)
        #print(f"Fields in radar data from {file_path}:")
        #print(list(radar.fields.keys()))
    except Exception as e:
        print(f"Failed to read radar data from {file_path}: {e}")

# Check the fields for the first file
if files:
    check_radar_fields(files[0])

# Function to read radar data
def read_radar_data(file_path):
    try:
        with fs.open(file_path, 'rb') as f:
            radar_data = f.read()
        radar_file = BytesIO(radar_data)
        radar = pyart.io.read_nexrad_archive(radar_file)
        #print(f"Successfully read radar data from {file_path}")
        return radar
    except Exception as e:
        print(f"Failed to read radar data from {file_path}: {e}")
        return None

# Function to find the nearest grid point to a given latitude and longitude
def find_nearest_grid_point(grid, lat, lon):
    lat_diff = np.abs(grid.point_latitude['data'][0] - lat)
    lon_diff = np.abs(grid.point_longitude['data'][0] - lon)
    return np.unravel_index((lat_diff + lon_diff).argmin(), lat_diff.shape)

# Collect data for the NetCDF files
time_height_data_tower = []
time_height_data_smc = []
times = []
heights = None  # Initialize heights to None

# Initialize convective feature duration variables in minutes
convective_duration_tower = 0
convective_duration_smc = 0

previous_time = None

for file in files:
    radar = read_radar_data(file)
    if radar is None:
        continue

    # Apply despeckle filter to the reflectivity field
    pyart.correct.despeckle_field(radar, 'reflectivity', threshold=-100, size=100, delta=0.1)
    gatefilter = pyart.correct.GateFilter(radar)
    gatefilter.exclude_below('reflectivity', 10)

    # Append the corresponding time once per file
    radar_time = pd.to_datetime(radar.time['units'].split('since ')[-1]) + pd.to_timedelta(radar.time['data'][0], unit='s')
    times.append(np.datetime64(radar_time))

    # Calculate the time interval between consecutive scans
    if previous_time is not None:
        time_interval = (radar_time - previous_time).total_seconds() / 60  # Convert to minutes
    else:
        time_interval = 0  # Default to 5 minutes if it's the first scan
    previous_time = radar_time

    for idx in [0, 1]:
        # Extract vertical profile
        profile = pyart.util.column_vertical_profile(radar, latitude[idx], longitude[idx], azimuth_spread=1, spatial_spread=1)
        reflectivity = profile['reflectivity']

        # Get the heights of the sweeps if not already obtained
        if heights is None:
            heights = radar.gate_altitude['data'][0, :]

        # Append the profile to the respective array
        if idx == 0:
            time_height_data_tower.append(reflectivity)
        else:
            time_height_data_smc.append(reflectivity)

    # Feature detection
    grid = pyart.map.grid_from_radars(
        (radar,),
        grid_shape=(1, 401, 401),  # Increase the grid resolution by doubling the number of points
        grid_limits=((0, 10000), (-50000.0, 50000.0), (-50000.0, 50000.0)),
        fields=['reflectivity'],
    )
    dx = grid.x['data'][1] - grid.x['data'][0]
    dy = grid.y['data'][1] - grid.y['data'][0]
    feature_dict = pyart.retrieve.feature_detection(
        grid,
        dx,
        dy,
        field='reflectivity',
        always_core_thres=40,
        bkg_rad_km=20,
        use_cosine=True,
        max_diff=5,
        zero_diff_cos_val=55,
        weak_echo_thres=10,
        max_rad_km=2,
    )
    feature_masked = np.ma.masked_equal(feature_dict['feature_detection']['data'], 0)
    feature_masked = np.ma.masked_equal(feature_masked, 3)
    feature_dict['feature_detection']['data'] = feature_masked
    grid.add_field('feature_detection', feature_dict['feature_detection'], replace_existing=True)

    # Check for convective feature detection at the locations
    tower_grid_point = find_nearest_grid_point(grid, latitude[0], longitude[0])
    smc_grid_point = find_nearest_grid_point(grid, latitude[1], longitude[1])

    print(f"Tower Grid Point: {tower_grid_point}, Feature Value: {feature_masked[0, tower_grid_point[0], tower_grid_point[1]]}")
    print(f"SMC Grid Point: {smc_grid_point}, Feature Value: {feature_masked[0, smc_grid_point[0], smc_grid_point[1]]}")

    if feature_masked[0, tower_grid_point[0], tower_grid_point[1]] == 2:
        convective_duration_tower += time_interval  # Add the actual time interval

    if feature_masked[0, smc_grid_point[0], smc_grid_point[1]] == 2:
        convective_duration_smc += time_interval  # Add the actual time interval

# Determine the maximum profile length
max_length_tower = max(len(profile) for profile in time_height_data_tower)
max_length_smc = max(len(profile) for profile in time_height_data_smc)

# Pad profiles with NaNs to match the maximum length
padded_time_height_data_tower = np.full((len(time_height_data_tower), max_length_tower), np.nan)
padded_time_height_data_smc = np.full((len(time_height_data_smc), max_length_smc), np.nan)

for i, profile in enumerate(time_height_data_tower):
    padded_time_height_data_tower[i, :len(profile)] = profile
for i, profile in enumerate(time_height_data_smc):
    padded_time_height_data_smc[i, :len(profile)] = profile

# Convert collected data to 2D arrays
time_height_data_tower = np.array(padded_time_height_data_tower)
time_height_data_smc = np.array(padded_time_height_data_smc)

# Ensure heights array matches the padded data's second dimension
heights_tower = np.linspace(heights[0], heights[-1], max_length_tower)
heights_smc = np.linspace(heights[0], heights[-1], max_length_smc)

# Create the xarray Datasets
ds_tower = xr.Dataset(
    {
        "reflectivity": (["time", "height"], time_height_data_tower)
    },
    coords={
        "time": times[:len(time_height_data_tower)],  # Ensure time dimension matches the data
        "height": heights_tower
    }
)

ds_smc = xr.Dataset(
    {
        "reflectivity": (["time", "height"], time_height_data_smc)
    },
    coords={
        "time": times[:len(time_height_data_smc)],  # Ensure time dimension matches the data
        "height": heights_smc
    }
)

# Save the datasets to NetCDF files
output_file_tower = "combined_radar_Tower2.nc"
output_file_smc = "combined_radar_SMC2.nc"
ds_tower.to_netcdf(output_file_tower)
ds_smc.to_netcdf(output_file_smc)
#print(f"Saved combined dataset to {output_file_tower}")
#print(f"Saved combined dataset to {output_file_smc}")

### Start of the second part of the code

# Load the NetCDF files
ds_tower = xr.open_dataset(output_file_tower)
ds_smc = xr.open_dataset(output_file_smc)

# Extract data from the datasets
times = ds_tower['time'].values
heights_tower = ds_tower['height'].values
reflectivity_tower = ds_tower['reflectivity'].values

heights_smc = ds_smc['height'].values
reflectivity_smc = ds_smc['reflectivity'].values

# Convert to numpy arrays
padded_time_height_data_tower = np.array(reflectivity_tower)
padded_time_height_data_smc = np.array(reflectivity_smc)

# Replace NaN values with zeros for Z-R calculation
reflectivity_tower = np.nan_to_num(padded_time_height_data_tower)
reflectivity_smc = np.nan_to_num(padded_time_height_data_smc)

# Calculate rainfall rate using Z-R relationship (for the lowest height level)
Z_tower = 10 ** (reflectivity_tower / 10)  # Convert dBZ to Z
R_tower = (Z_tower / 300) ** (1 / 1.4)  # Calculate rainfall rate

Z_smc = 10 ** (reflectivity_smc / 10)  # Convert dBZ to Z
R_smc = (Z_smc / 300) ** (1 / 1.4)  # Calculate rainfall rate

# Initialize total rainfall variables
total_rainfall_tower = 0
total_rainfall_smc = 0

### Plotting
frames_dir = 'frames/Vertical'
os.makedirs(frames_dir, exist_ok=True)
frames = []

# Prepare the meshes for plotting
time_mesh_smc, height_mesh_smc = np.meshgrid(mdates.date2num(times), heights_smc)
time_mesh_tower, height_mesh_tower = np.meshgrid(mdates.date2num(times), heights_tower)

# Initialize empty arrays for the gradual addition of data
incremental_data_smc = np.full_like(padded_time_height_data_smc, np.nan)
incremental_data_tower = np.full_like(padded_time_height_data_tower, np.nan)

for i in range(len(files)):
    fig = plt.figure(figsize=(20, 15))  # Adjust the figsize to accommodate the new subplot

    # Plot map with radar data
    radar = read_radar_data(files[i])
    if radar is None:
        continue
    ax1 = fig.add_axes([0.05, 0.5, 0.4, 0.45], projection=ccrs.PlateCarree())  # Adjust position and size
    radar_display = pyart.graph.RadarMapDisplay(radar)
    try:
        radar_display.plot_ppi_map('reflectivity',
                             sweep=0,
                             vmin=10,
                             vmax=65,
                             ax=ax1,
                             title=f'Reflectivity for {station} {times[i]}',
                             cmap='pyart_HomeyerRainbow',
                             colorbar_flag=False)  # Disable the built-in colorbar
        mappable = radar_display.plots[0]
    except Exception as e:
        print(f"Error plotting radar data for file {file}: {e}")
        plt.close(fig)
        continue

    ax1.set_xlim(-88.2, -87.7)
    ax1.set_ylim(41.5, 41.9)
    ax1.add_feature(USCOUNTIES, linewidth=0.5)
    for lat, lon, label, marker, color in zip(latitude, longitude, labels, markers, colors):
        ax1.plot(lon, lat, marker, label=label, color=color, transform=ccrs.PlateCarree(), markersize=10)
    ax1.legend(loc='upper right', fontsize='large', title='Locations')
    cbar = plt.colorbar(mappable, ax=ax1, orientation='horizontal', fraction=0.046, pad=0.04)
    cbar.set_label('Equivalent Reflectivity Factor (dBZ)')

    # Feature detection
    grid = pyart.map.grid_from_radars(
        (radar,),
        grid_shape=(1, 401, 401),  # Increase the grid resolution by doubling the number of points
        grid_limits=((0, 10000), (-50000.0, 50000.0), (-50000.0, 50000.0)),
        fields=['reflectivity'],
    )
    dx = grid.x['data'][1] - grid.x['data'][0]
    dy = grid.y['data'][1] - grid.y['data'][0]
    feature_dict = pyart.retrieve.feature_detection(
        grid,
        dx,
        dy,
        field='reflectivity',
        always_core_thres=40,
        bkg_rad_km=20,
        use_cosine=True,
        max_diff=5,
        zero_diff_cos_val=55,
        weak_echo_thres=10,
        max_rad_km=2,
    )
    feature_masked = np.ma.masked_equal(feature_dict['feature_detection']['data'], 0)
    feature_masked = np.ma.masked_equal(feature_masked, 3)
    feature_dict['feature_detection']['data'] = feature_masked
    grid.add_field('feature_detection', feature_dict['feature_detection'], replace_existing=True)

    # New ax6 for feature detection
    ax6 = fig.add_axes([0.05, 0.05, 0.4, 0.4], projection=ccrs.PlateCarree())  # Adjust position and size
    grid_display = pyart.graph.GridMapDisplay(grid)
    grid_display.plot_grid(
        'feature_detection',
        vmin=0,
        vmax=2,
        cmap=plt.get_cmap('viridis', 3),
        ax=ax6,
        transform=ccrs.PlateCarree(),
        ticks=[1 / 3, 1, 5 / 3],
        ticklabs=['', 'Stratiform', 'Convective'],
    )

    ax6.set_xlim(-88.2, -87.7)
    ax6.set_ylim(41.5, 41.9)
    ax6.add_feature(USCOUNTIES, linewidth=0.5)
    for lat, lon, label, marker, color in zip(latitude, longitude, labels, markers, colors):
        ax6.plot(lon, lat, marker, label=label, color=color, transform=ccrs.PlateCarree(), markersize=10)
    ax6.legend(loc='upper right', fontsize='large', title='Locations')

    # Check for convective feature detection at the locations
    tower_grid_point = find_nearest_grid_point(grid, latitude[0], longitude[0])
    smc_grid_point = find_nearest_grid_point(grid, latitude[1], longitude[1])

    if feature_masked[0, tower_grid_point[0], tower_grid_point[1]] == 2:
        convective_duration_tower += time_interval  # Add the actual time interval

    if feature_masked[0, smc_grid_point[0], smc_grid_point[1]] == 2:
        convective_duration_smc += time_interval  # Add the actual time interval

    # Create a time-height profile plot for SMC
    ax2 = fig.add_axes([0.55, 0.7, 0.4, 0.2])
    c_smc = ax2.pcolormesh(time_mesh_smc, height_mesh_smc, incremental_data_smc.T, shading='auto', cmap='pyart_HomeyerRainbow', vmin=0, vmax=65)
    fig.colorbar(c_smc, ax=ax2, label='Reflectivity (dBZ)')
    ax2.set_xlim(time_mesh_smc.min(), time_mesh_smc.max())
    ax2.set_ylim(height_mesh_smc.min(), height_mesh_smc.max())
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Height (ft)')
    ax2.xaxis_date()
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax2.set_title(f'Reflectivity Profile Above {labels[1]} Site')
    fig.autofmt_xdate()

    # Create a time-height profile plot for Tower
    ax3 = fig.add_axes([0.55, 0.4, 0.4, 0.2])
    c_tower = ax3.pcolormesh(time_mesh_tower, height_mesh_tower, incremental_data_tower.T, shading='auto', cmap='pyart_HomeyerRainbow', vmin=0, vmax=65)
    fig.colorbar(c_tower, ax=ax3, label='Reflectivity (dBZ)')
    ax3.set_xlim(time_mesh_tower.min(), time_mesh_tower.max())
    ax3.set_ylim(height_mesh_tower.min(), height_mesh_tower.max())
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Height (ft)')
    ax3.xaxis_date()
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax3.set_title(f'Reflectivity Profile Above {labels[0]} Site')
    fig.autofmt_xdate()

    # Calculate the time interval in hours
    if i == 0:
        time_interval = 0
    else:
        time_interval = (times[i] - times[i-1]).astype('timedelta64[s]').astype(float) / 3600

    # Create a rainfall rate plot for SMC
    ax4 = fig.add_axes([0.55, 0.1, 0.4, 0.2])
    ax4.plot(times[:i+1], R_smc[:i+1, 0], color='blue')
    ax4.set_xlim(times[0], times[-1])
    ax4.set_ylim(-5, 90)
    ax4.set_yticks(np.arange(0, 91, 10))
    ax4.set_yticklabels([f'{int(y)}' for y in np.arange(0, 91, 10)])
    ax4.set_xlabel('Time')
    ax4.set_ylabel('Rainfall Rate (mm/hr)')
    ax4.xaxis_date()
    ax4.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax4.set_title(f'Rainfall Rate at Ground Level ({labels[1]})')
    fig.autofmt_xdate()

    # Update total rainfall for SMC
    if i > 0:
        total_rainfall_smc += R_smc[i, 0] * time_interval

    # Add total rainfall annotation
    ax4.annotate(f'Total: {total_rainfall_smc:.2f} mm', xy=(.993, 0.890), xycoords='axes fraction', fontsize=12,
                 horizontalalignment='right', verticalalignment='bottom', bbox=dict(facecolor='white', alpha=0.6))

    # Create a rainfall rate plot for Tower
    ax5 = fig.add_axes([0.55, -0.2, 0.4, 0.2])
    ax5.plot(times[:i+1], R_tower[:i+1, 0], color='blue')
    ax5.set_xlim(times[0], times[-1])
    ax5.set_ylim(-5, 90)
    ax5.set_yticks(np.arange(0, 91, 10))
    ax5.set_yticklabels([f'{int(y)}' for y in np.arange(0, 91, 10)])
    ax5.set_xlabel('Time')
    ax5.set_ylabel('Rainfall Rate (mm/hr)')
    ax5.xaxis_date()
    ax5.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax5.set_title(f'Rainfall Rate at Ground Level ({labels[0]} Site)')
    fig.autofmt_xdate()

    # Update total rainfall for Tower
    if i > 0:
        total_rainfall_tower += R_tower[i, 0] * time_interval

    # Add total rainfall annotation
    ax5.annotate(f'Total: {total_rainfall_tower:.2f} mm', xy=(.993, 0.893), xycoords='axes fraction', fontsize=12,
                 horizontalalignment='right', verticalalignment='bottom', bbox=dict(facecolor='white', alpha=0.6))

    # Update data
    incremental_data_smc[:i+1, :] = padded_time_height_data_smc[:i+1, :]
    incremental_data_tower[:i+1, :] = padded_time_height_data_tower[:i+1, :]

    c_smc.set_array(incremental_data_smc.T.flatten())
    c_tower.set_array(incremental_data_tower.T.flatten())

    plt.tight_layout()
    frame_filename = os.path.join(frames_dir, f'2combined_frame_{i}.png')
    plt.savefig(frame_filename, bbox_inches='tight')
    frames.append(frame_filename)
    plt.close(fig)

    #print(f"Saved frame {i+1}/{len(files)}: {frame_filename}")

# Create animated GIF
images = [Image.open(frame) for frame in frames]
gif_filename = 'June23rdEvent.gif'
images[0].save(gif_filename, save_all=True, append_images=images[1:], duration=375, loop=0)

# Cleanup
for frame in frames:
    os.remove(frame)

# Close and delete the NetCDF files
ds_tower.close()
ds_smc.close()
os.remove(output_file_tower)
os.remove(output_file_smc)

print(f'Animated GIF saved as {gif_filename}')
with open(gif_filename, 'rb') as f:
    display(IPImage(data=f.read(), format='gif'))

# Print convective feature durations
print(f"Tower site experienced convective features for {convective_duration_tower} minutes")
print(f"SMC site experienced convective features for {convective_duration_smc} minutes")


Tower Grid Point: (243, 229), Feature Value: --
SMC Grid Point: (244, 239), Feature Value: --
Tower Grid Point: (243, 229), Feature Value: 1.0
SMC Grid Point: (244, 239), Feature Value: --
Tower Grid Point: (243, 229), Feature Value: 1.0
SMC Grid Point: (244, 239), Feature Value: --
Tower Grid Point: (243, 229), Feature Value: 1.0
SMC Grid Point: (244, 239), Feature Value: --
Tower Grid Point: (243, 229), Feature Value: 1.0
SMC Grid Point: (244, 239), Feature Value: --
Tower Grid Point: (243, 229), Feature Value: 1.0
SMC Grid Point: (244, 239), Feature Value: --
Tower Grid Point: (243, 229), Feature Value: --
SMC Grid Point: (244, 239), Feature Value: --
Tower Grid Point: (243, 229), Feature Value: --
SMC Grid Point: (244, 239), Feature Value: --
Tower Grid Point: (243, 229), Feature Value: --
SMC Grid Point: (244, 239), Feature Value: --
Tower Grid Point: (243, 229), Feature Value: --
SMC Grid Point: (244, 239), Feature Value: --
Tower Grid Point: (243, 229), Feature Value: 1.0
SMC Gr